In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime as dt
import time as tm
import multiprocessing as mp

In [2]:
vehicle_sales_path = r'F:\DataSpell\Pandas_training\data_sets\car_prices.csv'
vehicle_sales = pd.read_csv(vehicle_sales_path, parse_dates=['year']).dropna(how='all')

In [3]:
# for col in vehicle_sales.select_dtypes(include=['int', 'float']).columns:
#   if vehicle_sales[col].dtype.kind == 'i':
#     vehicle_sales[col] = pd.to_numeric(vehicle_sales[col], downcast='integer')
#   else:
#     vehicle_sales[col] = pd.to_numeric(vehicle_sales[col], downcast='float')

vehicle_sales['state'] = vehicle_sales['state'].astype('category')
vehicle_sales['year'] = pd.to_datetime(vehicle_sales['year'], format='%Y').dt.year
vehicle_sales['make'] = vehicle_sales['make'].fillna('unknown').astype('category')
vehicle_sales['model'] = vehicle_sales['model'].fillna('unknown').astype(str)
vehicle_sales['trim'] = vehicle_sales['trim'].fillna('unknown').astype(str).str.lower()
vehicle_sales['body'] = vehicle_sales['body'].fillna('unknown').astype('category')
vehicle_sales['transmission'] = vehicle_sales['transmission'].fillna('unknown').astype('category')
vehicle_sales['vin'] = vehicle_sales['vin'].fillna('unknown')
vehicle_sales['condition'] = vehicle_sales['condition'].fillna(0)
vehicle_sales['odometer'] = vehicle_sales['odometer'].fillna(0)
vehicle_sales['color'] = vehicle_sales['color'].fillna('unknown').astype('category')
vehicle_sales['interior'] = vehicle_sales['interior'].fillna('unknown').astype('category')
vehicle_sales['mmr'] = vehicle_sales['mmr'].fillna(0)
vehicle_sales['sellingprice'] = vehicle_sales['sellingprice'].fillna(0)

# vehicle_sales['sellingprice'].apply(lambda number: number.is_integer())

In [4]:
# # Define a function to parse datetime strings and convert to Lithuania time
# def convert_to_lithuania_time(dt_str):
#     try:
#         # Parse the datetime string and convert to Lithuania time
#         dt = pd.to_datetime(dt_str).tz_convert('Europe/Vilnius')
#     except Exception:
#         # If parsing fails, return NaT
#         dt = pd.NaT
#     return dt
# 
# # Apply the function to the 'saledate' column
# vehicle_sales['saledate'] = vehicle_sales['saledate'].apply(convert_to_lithuania_time)
# 
# # Replace NaT values with a specific date (e.g., '1900-01-01')
# vehicle_sales['saledate'] = vehicle_sales['saledate'].fillna(pd.Timestamp('1900-01-01', tz='Europe/Vilnius'))
# 
# # Check the data type of the 'saledate' column
# vehicle_sales['saledate'].dtypes

In [5]:
vehicle_sales

In [6]:
vehicle_sales.nunique()

In [7]:
# Get the current time
start_time = tm.time()

# Load your script here
vehicle_sales = pd.read_csv(vehicle_sales_path)

# Get the current time again
end_time = tm.time()

# Calculate the time it took to load your script
load_time = end_time - start_time

print(f"The script took {load_time} seconds to load.") # 1.0112128257751465 seconds

In [8]:
vehicle_sales.info() # 68.2+ MB

In [9]:
vehicle_sales.dtypes

# Upper is data cleaning handling to reduce memory usage and data types

In [10]:
vehicle_sales

In [11]:
vehicle_sales.groupby(['make', 'trim', 'year'], observed=True)['sellingprice'].agg(
  ['sum', 'count']
)

In [12]:
# Filter out rows where 'make' is 'unknown'
filtered_vehicle_sales = vehicle_sales[vehicle_sales['make'] != 'unknown']

# Create the pivot table on the filtered data
pivot_table = filtered_vehicle_sales.pivot_table(columns='make', index='year', values='sellingprice',
                                                 aggfunc='sum', observed=True, margins=True, margins_name='Total').dropna(how='all')
pivot_table

In [13]:
vehicle_sales_sell_price_quartiles = pd.qcut(vehicle_sales['sellingprice'], q=4, labels=['Q1', 'Q2', 'Q3', 'Q4'])
vehicle_sales.insert(15, 'Sell price quartiles', vehicle_sales_sell_price_quartiles)

In [14]:
highest_quartile_vehicles = vehicle_sales[vehicle_sales['Sell price quartiles'] == 'Q4']
vehicle_sales

In [15]:
vehicle_sales['sellingprice'].idxmax()

In [16]:
cross_tab = pd.crosstab(vehicle_sales['make'], vehicle_sales['body'], margins=True, margins_name='Total')
cross_tab

In [17]:
bins = [0, 10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000]
labels = ['0-10k', '10-20k', '20-30k', '30-40k', '40-50k', '50-60k', '60-70k', '70-80k', '80-90k', '90-100k']

pd_cut_test = pd.cut(vehicle_sales['sellingprice'], bins=bins, labels=labels)
pd_cut_test

In [18]:
rolling_test = vehicle_sales['sellingprice'].rolling(window=4, min_periods=1).sum()
rolling_test

In [19]:
corr_testing = vehicle_sales[['sellingprice', 'odometer', 'mmr']].corr()
corr_testing

In [20]:
vehicle_sales[['mmr', 'sellingprice']]
vehicle_sales

In [21]:
vehicle_sales['mmr'].quantile(0.25)
vehicle_sales['mmr'].quantile(0.50)
vehicle_sales['mmr'].quantile(0.75)

In [22]:
vehicle_sales['sellingprice'].ewm(span=10, min_periods=1).sum()

# Exercises

# 1. Load the dataset and display the first 10 rows.

In [23]:
car_sales = pd.read_csv(r'F:\DataSpell\Pandas_training\data_sets\car_prices.csv')
car_sales.head(10)

# 2. Display the last 10 rows of the dataset.

In [24]:
car_sales.tail(10)

# 3. Check for any missing values in the dataset.

In [25]:
car_sales.isna().sum()
car_sales.isnull().sum()
car_sales.info()
car_sales.isnull().any()
car_sales.isnull().all()

# 4. Find the number of unique makes, models, and trims.

In [26]:
car_sales

In [27]:
car_sales['make'].unique()
car_sales['model'].unique()
car_sales['trim'].unique()
# other solution
car_sales['make'].nunique()
car_sales['model'].nunique()
car_sales['trim'].nunique()

# 5. Convert the saledate column to a datetime object.

In [28]:
# %%timeit
car_sales

In [29]:
car_sales['saledate'] = pd.to_datetime(car_sales['saledate'], errors='coerce', utc=True)
car_sales['saledate'] = car_sales['saledate'].dt.tz_convert('Europe/Vilnius')
car_sales.dtypes

# 6. Extract the year and month from the saledate column into separate columns.

In [30]:
# year = car_sales['saledate'].pd.Timestamp
year = car_sales['saledate'].apply(lambda y: pd.Timestamp(y).year)
month = car_sales['saledate'].apply(lambda m: pd.Timestamp(m).month)
month[month.isna()]
month

# 7. Filter the dataset to only include cars with an odometer reading less than 100,000 miles.

In [31]:
car_sales.query('odometer > 100_000')

# 8. Create a new column indicating whether the car's transmission is automatic or manual.

In [32]:
column_names = list(car_sales.columns)
column_names_index = {index: value for value, index in enumerate(column_names)}
column_names_index

In [33]:
def car_transmission(row):
  transmission = row['transmission']
  if transmission == 'automatic':
    return 'Automatic'
  if transmission == 'manual':
    return 'Manual'
  else:
    return 'Unknown'

car_sales_transmission = car_sales.apply(car_transmission, axis=1)
# car_sales.insert(6, 'car_transmission', car_sales_transmission)
car_sales

# 9. Group the data by year and calculate the average selling price for each year.

In [34]:
year = car_sales['saledate'].dt.year
year_month = car_sales['saledate'].dt.to_period('M')
# car_sales.groupby(year)['sellingprice'].mean()
car_sales.groupby(year_month)['sellingprice'].mean()

# 10. Identify the top 5 most common car colors.

In [35]:
car_sales['color'].value_counts().head(5)

# 11. Calculate the average odometer reading by make and model.

In [36]:
car_sales.groupby(['make', 'model'])['odometer'].mean().sort_values(ascending=False)

# 12. Find the make and model of the car with the highest selling price.

In [37]:
car_sales.groupby(['make', 'model'])['sellingprice'].max().sort_values(ascending=False)

In [38]:
index_of_max_sales = car_sales['sellingprice'].idxmax()
vehicle_sales.loc[index_of_max_sales, ['make', 'model']]

# 13. Filter the dataset to include only cars from states with more than 50 listings in the dataset.

In [39]:
states_count = car_sales['state'].value_counts() 
states_over_50 = states_count[states_count > 50].index
car_sales[car_sales['state'].isin(states_over_50)]

# 14. Create a pivot table showing the average selling price of cars by make and body type.

In [40]:
car_sales.pivot_table(index='make', 
                      columns='body', 
                      values='sellingprice', 
                      aggfunc='mean',
                      margins=True,
                      margins_name='Average Total'
                      )

# 15. Calculate the difference between the MMR (Market Market Range) and the selling price, then create a column for this difference.

In [41]:
car_sales_columns= list(car_sales.columns)
index_columns = {index: value for index, value in enumerate(car_sales_columns)}
index_columns

In [42]:
difference = car_sales['mmr'] - car_sales['sellingprice']
# car_sales.insert(15, "mmr sell price diff", difference)
car_sales

# 16. Identify cars with a condition rating above 4.0 and an odometer reading less than 50,000 miles.

In [43]:
car_sales.query('condition > 4 & odometer < 50_000')

# 17. Calculate the average selling price of cars for each combination of make and body type.

In [44]:
car_sales.groupby(['make', 'body'])['sellingprice'].mean()

# 18. Determine the make and model with the highest average odometer reading.

In [45]:
odometer_max = car_sales['odometer'].idxmax()
car_odometer_max = car_sales.loc[odometer_max, ['make', 'model']]
car_odometer_max

In [46]:
car_sales.groupby(['make', 'model'])['odometer'].mean().sort_values(ascending=False).idxmax()

# 19. Drop the vin column from the dataset.

In [47]:
drop_vin_column = car_sales.drop('vin', axis=1)
drop_vin_column

# 20. Rename the mmr column to market_market_range.

In [49]:
car_sales = car_sales.rename(columns={'mmr': 'market_market_range'})
car_sales

# 21. Find the year with the highest number of car listings.

In [55]:
# car_sales.groupby('year')['make'].count().sort_values(ascending=False)

car_sales['year'].value_counts().idxmax()

# 22. Create a histogram (text-based, counts per bin) of the odometer readings, dividing the range into bins of 10,000 miles.

In [59]:
bins = list(range(0, int(car_sales['odometer'].max() + 10000), 10000))

car_sales['odometer_bins'] = pd.cut(car_sales['odometer'], bins=bins)

histogram = car_sales['odometer_bins'].value_counts().sort_index()
histogram

# 23. For each make, count the number of cars with transmission as automatic.

In [70]:
automatic_transmission = car_sales[car_sales['transmission'] == 'automatic']
automatic_transmission.groupby('make')['transmission'].count().sort_values(ascending=False)

In [78]:
# car_sales.pivot_table(index='make', columns='transmission', aggfunc='count', margins=True, margins_name='Total')
count_transmission = car_sales.pivot_table(index='make', columns='transmission', aggfunc='size')

count_transmission['automatic'].sort_values(ascending=False)

# 24. Replace any instance of "inc" in the seller's name with "Inc."

In [87]:
car_seller = car_sales['seller']
car_seller.str.replace('inc', 'Inc.')

# 25. Split the saledate column into separate columns for the date and time of sale.

In [93]:
# sale_date = car_sales['saledate'].dt.to_period('D')
sale_date = car_sales['saledate'].dt.date
sales_time = car_sales['saledate'].dt.time
sale_date

# 26. Identify and remove any duplicate rows, based on vin.

In [99]:
unique_vin = car_sales['vin'].drop_duplicates()
unique_vin_sales = car_sales[car_sales['vin'].isin(unique_vin)]

car_sales.drop_duplicates(subset='vin')

# 27. Apply a function to normalize the condition ratings to a scale of 1 to 10.

In [96]:
# Calculate the minimum and maximum of the 'condition' column
min_condition = car_sales['condition'].min()
max_condition = car_sales['condition'].max()

# Apply the min-max normalization formula
car_sales['condition_normalized'] = ((car_sales['condition'] - min_condition) / (max_condition - min_condition)) * 9 + 1

# 28. Find the average condition rating for each make.

In [102]:
car_sales.groupby('make')['condition'].mean().sort_values(ascending=False)

# 29. Use boolean indexing to filter cars that are either black or white.

In [105]:
car_sales[car_sales['color'].isin(['black', 'white'])]

# 30. Write a function that categorizes cars into "low", "medium", and "high" mileage categories based on the odometer reading and apply it to create a new column.

In [120]:
# car_sales['odometer'].value_counts(bins=10, normalize=True) * 100
car_sales['odometer'].value_counts(bins=3)

In [113]:
def car_odometer(row):
  odometer = row['odometer']
  if odometer <= 333333.667:
    return 'low'
  if 333333.667 < odometer <= 666666.333:
    return 'medium'
  if odometer > 666666.333:
    return 'high'

car_sales.apply(car_odometer, axis=1).value_counts()